In [6]:
import pandas as pd
df = pd.read_csv("/kaggle/input/partiality-scores/labeled.csv")
df.head()

,tweet_id,tweet_text,partiality_score,preprocessed tweets,label
0,1641953510903234562,روس تحتج علي تصرف استفزازيه قوات امريكيه في سور,4.50,روس تحتج عل تصرف استفزاز قوا امريك في سور,neutral
1,1641953254094381057,روس تحتج علي استفزاز قوات اميركيه في سور,4.50,روس تحتج عل استفزاز قوا اميرك في سور,neutral
2,1641953209810862080,توضيح لن يحدث شي مهم هذا اسبوع ومن متوقع تبدا ...,-5.25,توضيح لن يحدث شي مهم هذا اسبوع ومن متوقع تبد ا...,pro russia
3,1641953161295347716,فرنسا بتشوف ايام شرق اوسطيه افريقيه زي الفل يع...,-5.25,رنس تشوف ايام شرق اوسط افريق زي الفل يعن رييس ...,pro russia
4,1641952955136974848,روس تحتج علي استفزاز قوات اميركيه في سور سكاي ...,4.50,روس تحتج عل استفزاز قوا اميرك في سور سكا نيوز عرب,neutral


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets-sg-100/tweets_sg_100.trainables.syn1neg.npy
/kaggle/input/tweets-sg-100/tweets_sg_100
/kaggle/input/tweets-sg-100/tweets_sg_100.wv.vectors.npy
/kaggle/input/best_weightss/keras/best/1/pretrained_weights.weights.h5
/kaggle/input/best_weightss/keras/best/1/9385acc_100.keras
/kaggle/input/partiality-scores/labeled.csv
/kaggle/input/partiality-scores/label.csv
/kaggle/input/partiality-scores/unlabeled_tweets.csv


In [8]:
df2 = pd.read_csv("/kaggle/input/partiality-scores/unlabeled_tweets.csv")
df2.head()

,Unnamed: 0,tweets
0,0,صين هند روس من اشرس اعداء اسلام لعن الله
1,1,الله مع الحق تحي روس عظم قيصر وتن
2,2,زار دفاع روس تعليق حرك سفن عبر ممر امن محدد مو...
3,3,هه هه بايد راح يغادر بيت ابيض وبو جالس يضرب اوكر
4,4,من حق زيلينسك ان يدوس عل رقاب ادار امريك دول غ...


In [9]:
labeled_features = df['preprocessed tweets'].astype(str)
labels = df['label'].astype(str)

In [10]:
# spliiting labeled into train and test
labeled_features_train = list(labeled_features[:150000])
labeled_features_test = list(labeled_features[20000:])
labels_train = list(labels[:150000])
labels_test = list(labels[200000:])

In [11]:
unlabeled_features = df2['tweets'].astype(str)

In [12]:
import numpy as np
np.random.choice(4, size=1, replace=True)[0]

0

In [13]:
# loading pretrained embedder
from nltk.stem.isri import ISRIStemmer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import re

tweets_sg_100 = '/kaggle/input/tweets-sg-100/tweets_sg_100'
sg_model = gensim.models.Word2Vec.load(tweets_sg_100)

In [14]:
# adding important words to the embedder, words he didnt know about

# important words not recognized by the embedder 
russia = ['روسي','روس','روسيه','لروسي','والروس','ياروسي','روسی','موسك'] # 'روسيا'
belarus = ['يلاروسي','يلاروس','بيلاروس','بيلاروسيه','لبيلاروسي','والبيلاروس'] # 'بيلاروسيا'
ukr = ['اوكران','واوكراني','اوكر','لاوكراني','ناتو'] # 'اوكرانيا'
russian_hashtags = ['جيشالروس','قواتالروسيه','روسياليوم','روسياتنتصر','خارجيهالروسيه','عمليهالعسكريهالروسيه'] # روسيا
ukr_russian_hashtags = ['حربالروسيهالاوكرانيه','روسياوكراني','اوكرانياروسي','غزوالروسيلاوكرانيا','روسياواوكراني'] # 'حرب'
ukr_hashtags = ['روسياتغزواوكراني','روسياتكذب','روسيالص','روسياوالص','روسيامجرم','مجدلاوكرانيا','مخابراتالاوكرانيه'] # 'اوكرانيا'
zelensky = ['زيلينسك','زلينسك'] # 'اوباما'
putin = ['بوتن','وتن','پوتن','وتين','بوتين','پوتين'] #'بوتين'
war = ['حربالعالميه', 'حربالعالميهالثالثه','عسكريه'] # 'حرب'
egypt = ['مصر','مصريه','لمصر'] # 'عربي'
us = ['وامريك','الولاياتالمتحده','الولاياتالمتحده','الولاياتالمتحدهالامريكيه','البريطانيا','والولايا','امريك'] # 'امريكا'
gb = ['ريطاني'] # "بريطانيا"


russia.extend(russian_hashtags)
ukr.extend(ukr_hashtags)
war.extend(ukr_russian_hashtags)

added_words = []
added_words.extend(russia)
added_words.extend(ukr)
added_words.extend(war)
added_words.extend(putin)
added_words.extend(zelensky)
added_words.extend(egypt)
added_words.extend(us)
added_words.extend(gb)
added_words = set(added_words)
russia = set(russia)
ukr = set(ukr)
war = set(war)
putin = set(putin)
zelensky = set(zelensky)
egypt = set(egypt)
us = set(us)
gb = set(gb)

# add words that donot exist to wv
# russia 'روسيا', ukr 'اوكرانيا', war 'حرب' , putin 'بوتين', zelensky 'اوباما', belarus 'بيلاروسيا' , egypt 'قطر', us 'امريكا', gb "بريطانيا"
# for each set of words assign them to a vector of a similar word to them.
for word in russia:
    sg_model.wv.add_vector(word,sg_model.wv['روسيا'])
for word in ukr:
    sg_model.wv.add_vector(word,sg_model.wv['اوكرانيا'])
for word in war:
    sg_model.wv.add_vector(word,sg_model.wv['حرب'])
for word in putin:
    sg_model.wv.add_vector(word,sg_model.wv['بوتين'])
for word in zelensky:
    sg_model.wv.add_vector(word,sg_model.wv['اوباما'])
for word in belarus:
    sg_model.wv.add_vector(word,sg_model.wv['بيلاروسيا'])
for word in egypt:
    sg_model.wv.add_vector(word,sg_model.wv['قطر'])   
for word in us:
    sg_model.wv.add_vector(word,sg_model.wv['امريكا']) 
for word in gb:
    sg_model.wv.add_vector(word,sg_model.wv["بريطانيا"])

/opt/conda/lib/python3.10/site-packages/gensim/models/keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


In [15]:
# how many tweets 
ex_length = 150000
# how many words per tweet : pad with zeros if words are less than max_len
max_len = 64
# each word is represented in a vector of size 100 
word_vector = 100
X_train_labeled = np.zeros([ex_length,max_len,word_vector])

In [16]:
for ex in range(ex_length):
    words = labeled_features[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv:
            X_train_labeled[ex,position,:] = sg_model.wv[word]
        position += 1

In [31]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Map labels to integer values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(labels_train[:150000])

y_train_one_hot = to_categorical(y_train_encoded)

In [18]:
# how many tweets 
ex_length = 4000 #len(unlabeled_features)
# how many words per tweet : pad with zeros if words are less than max_len
max_len = 64
# each word is represented in a vector of size 100 
word_vector = 100
X_unlabeled = np.zeros([ex_length,max_len,word_vector])

In [19]:
for ex in range(ex_length):
    words = unlabeled_features[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv:
            X_unlabeled[ex,position,:] = sg_model.wv[word]
        position += 1

In [20]:
X_train_labeled = X_train_labeled.astype(np.float32)
X_unlabeled = X_unlabeled.astype(np.float32)

In [21]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Dense, Flatten

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 100),name='e_inp') 
decoder_inputs = Input(shape=(None, 100),name='d_inp')
encoder = Bidirectional(LSTM(64, return_state=True),name='enc')
encoder_outputs, hf, hb, cf, cb = encoder(encoder_inputs)
encoder_states = [hf, hb, cf, cb]
decoder_lstm = Bidirectional(LSTM(64, return_sequences=True, return_state=True),name='dec')
decoder_outputs, dhf, dhb, dcf, dcb = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(100,name="dense_1")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ e_inp (InputLayer)  │ (None, None, 100) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ d_inp (InputLayer)  │ (None, None, 100) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc (Bidirectional) │ [(None, 128),     │     84,480 │ e_inp[0][0]       │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec (Bidirectional) │ [(None, None,     │     84,480 │ d_inp[0][0],      │
│                     │ 128), (None, 64), │            │ enc[0][1],        │
│                     │ (None, 64),       │            │ enc[0][2],        │
│                     │ (None, 64),       │            │ enc[0][3],        │
│                     │ (None, 64)]       │            │ enc[0][4]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 100) │     12,900 │ dec[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 181,860 (710.39 KB)

 Trainable params: 181,860 (710.39 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.load_weights("/kaggle/input/best_weightss/keras/best/1/pretrained_weights.weights.h5")

In [23]:
encoder = model.get_layer('enc')
config = encoder.get_config()
config['layer']['config']['return_state'] = False
config['layer']['config']['return_sequences'] = True

encoder_layer_new = Bidirectional(LSTM.from_config(config['layer']['config']), name='enc')

# Get the weights of the original encoder layer and set them to the new layer
encoder_layer_new.build((None,None,100))
encoder_layer_new.set_weights(encoder.get_weights()) 

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras.losses import CategoricalCrossentropy

# Define the model
model = Sequential()
# Add Bidirectional LSTM layers
model.add(encoder_layer_new)
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))

# Add Dense layer
model.add(Dense(units=20, activation='relu'))

# Add Softmax layer
model.add(Dense(units=3, activation='softmax'))

loss= CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="sum_over_batch_size",
    name="categorical_crossentropy",
)
# Compile the model
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ enc (Bidirectional)             │ ?                      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,480 (330.00 KB)

 Trainable params: 84,480 (330.00 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.load_weights("/kaggle/input/best_weightss/keras/best/1/9385acc_100.keras")

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 4 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=False>:

Layer 'lstm_cell' was never built and thus it doesn't have any variables. However the weights file lists 3 variables for this layer.
In most cases, this error indicates that either:

1. The layer is owned by a parent layer that implements a `build()` method, but calling the parent's `build()` method did NOT create the state of the child layer 'lstm_cell'. A `build()` method must create ALL state for the layer, including the state of any children layers.

2. You need to implement the `def build_from_config(self, config)` method on layer 'lstm_cell', to specify how to rebuild it during loading. In this case, you might also want to implement the method that generates the build config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the layer (i.e. its variables) upon deserialization.

List of objects t

In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint filepath with '.keras' extension
checkpoint_filepath = 'best_weights_crossview.keras'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                      save_best_only=True,
                                      monitor='val_accuracy',
                                      mode='max',  # 'max' for accuracy
                                      verbose=1)


In [27]:
drop_symbol = np.zeros(100)
consistency_lambda = 3

In [28]:
from keras.optimizers import Adam
loss_fn = CategoricalCrossentropy(from_logits=False,label_smoothing=0.0,axis=-1,reduction="sum_over_batch_size",name="categorical_crossentropy")
# Compile the model
optimizer= Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [29]:
from keras.optimizers import Adam
loss_fn = CategoricalCrossentropy(from_logits=False,label_smoothing=0.0,axis=-1,reduction="sum_over_batch_size",name="categorical_crossentropy")
# Compile the model
optimizer= Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
import tensorflow as tf
import numpy as np

In [ ]:
# Training loop
for epoch in range(10):
    # Train model on labeled data
    history = model.fit(X_train_labeled, y_train_one_hot,epochs=1,batch_size=32,validation_split=0.1,callbacks=[checkpoint_callback])
    
    # Open a GradientTape context to record operations for automatic differentiation
    with tf.GradientTape(watch_accessed_variables=True) as tape:
        # Compute predictions for labeled data
       
        # Generate four different views for each tweet in the unlabeled data by droping words
        unlabeled_sequences_views = []
        for sequence in X_unlabeled:
            unlabeled_sequences_views.append(sequence)
            for _ in range(3):
                cross_view = sequence.copy()  # Make a copy of the original sequence
                drop = np.random.choice(64, size=1, replace=False)[0]
                cross_view[drop] = drop_symbol
                unlabeled_sequences_views.append(cross_view)
        

        
        # Convert to numpy array and then to TensorFlow tensor
        unlabeled_sequences_views = np.array(unlabeled_sequences_views, dtype=np.float32)
        unlabeled_sequences_views = tf.convert_to_tensor(unlabeled_sequences_views)
        
        # Compute predictions for unlabeled data using the model
        batch_size = 20
        unlabeled_preds = []
        for i in range(0, len(unlabeled_sequences_views), batch_size):
            end = min(i + batch_size, len(unlabeled_sequences_views))
            batch_preds = model(unlabeled_sequences_views[i:end], training=True)
            unlabeled_preds.append(batch_preds)
        unlabeled_preds = tf.concat(unlabeled_preds, axis=0)


        # Compute consistency loss between predictions from different views
        consistency_loss_value = tf.reduce_mean(tf.abs(unlabeled_preds[::4]-unlabeled_preds[1::4])
                                                +tf.abs(unlabeled_preds[::4]-unlabeled_preds[2::4])
                                                +tf.abs(unlabeled_preds[::4]-unlabeled_preds[3::4])) * consistency_lambda
        # Compute total loss
        consistency_loss_value = tf.cast(consistency_loss_value, dtype=tf.float32)
        total_loss = consistency_loss_value 

    # Compute gradients
    gradients = tape.gradient(total_loss, model.trainable_variables)

    # Update optimizer's parameters based on gradients
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f'Epoch [{epoch + 1}/{10}], Consistency Loss: {consistency_loss_value.numpy():.4f}')

4219/4219 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9132 - loss: 0.2388
Epoch 1: val_accuracy improved from -inf to 0.93453, saving model to best_weights_crossview.keras
4219/4219 ━━━━━━━━━━━━━━━━━━━━ 85s 19ms/step - accuracy: 0.9132 - loss: 0.2388 - val_accuracy: 0.9345 - val_loss: 0.1906
Epoch [1/10], Consistency Loss: 0.0897
2227/4219 ━━━━━━━━━━━━━━━━━━━━ 35s 18ms/step - accuracy: 0.9571 - loss: 0.1225